<!DOCTYPE html>
<html>

<body>
    <h2><span style="color: lightblue;">Influx v3</span></h2>
    <h3><strong>Desc:</strong></h3>
    <ul>
        <li>Write metrics to Influx Database.</li>
        <li>Read Metrics from a bucket.</li>
    </ul>
    <h3>Comments:</h3>
    <ul>
        <li>This notebook is still in <strong>DRAFT</strong> mode.</li>
        <li>This notebook was developed for Influx cloud v3. Do NOT use it with v2</li>
    </ul>
</body>
</html>

## Pre Req
1. Open an Iflux DB cloud account. Can be the free one. https://www.influxdata.com/get-influxdb/    
2. Create a new Bucket. For ex. ```Tables_bucket```.   
   
The rest of the flow follows the built-in samples
1. Open the "Load Data" option of "Client" 
2. Select "Python"
3. Follow the instructions. Step "install dependencies" is pip install influxdb3-python, pip install pandas

## Get Token. 
Without a token, the client code can't call the API. To get a Token, log in to the cloud console, Load Data, API Tokens, Generate API Token. 

In [44]:
%%bash
export INFLUXDB_TOKEN=XE9AyZ-3y-HJNyupKWiLgzVo5JMew-Y31Vq7gbakekdP66wIkBslEdnyrCc-vQ0t9MGFj449z0LvFhepVOwFfw==

## Initialize Client
Copy the code. I named the organization in Influx "Dev"

Notice. The original code, proivded by Influx returns an error: ```[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)``` . To solve it, add the coomand ``` ssl_ca_cert=certifi.where()```. Resource: # https://stackoverflow.com/questions/69401104/influxdb-2-0-certificate-verify-failed-certificate-has-expired-ssl-c1129
   

In [9]:
import os, time
from influxdb_client_3 import InfluxDBClient3, Point
import certifi # we need it to support their certification problems. 

# Read from the OS or pass the parameter directly. 
token = os.environ.get("INFLUXDB_TOKEN")
token = "XE9AyZ-3y-HJNyupKWiLgzVo5JMew-Y31Vq7gbakekdP66wIkBslEdnyrCc-vQ0t9MGFj449z0LvFhepVOwFfw=="
print (token)
org = "Dev"
host = "https://us-east-1-1.aws.cloud2.influxdata.com"

client = InfluxDBClient3(host=host, token=token, org=org, ssl_ca_cert=certifi.where())

XE9AyZ-3y-HJNyupKWiLgzVo5JMew-Y31Vq7gbakekdP66wIkBslEdnyrCc-vQ0t9MGFj449z0LvFhepVOwFfw==


## Copy (Insert) Data  
The object "Bucket" in the UI called "database" in python.
The measurement called "Census". Notic the ```Point``` object uses it.  



In this data example, we have some important concepts:
- **measurement**: Primary filter for the thing you are measuring. Since we are measuring the sample census of insects, our measurement is "census".
- **tag**: Key-value pair to store metadata about your fields. We are storing the "location" of where each census is taken. Tags form part of your primary key.
- **field**:	Key-value pair that stores the actual data you are measuring.	We are storing the insect "species" and "count" as the key-value pair. Fields are not indexed and can be stored as integers, floats, strings, or booleans.

In [12]:
database="Tables_Bucket"

data = {
  "point1": {
    "location": "Klamath",
    "species": "bees",
    "count": 25,
  },
  "point2": {
    "location": "Portland",
    "species": "ants",
    "count": 32,
  },
  "point3": {
    "location": "Klamath",
    "species": "bees",
    "count": 28,
  },
  "point4": {
    "location": "Portland",
    "species": "ants",
    "count": 36,
  },
  "point5": {
    "location": "Klamath",
    "species": "bees",
    "count": 27,
  },
  "point6": {
    "location": "Portland",
    "species": "ants",
    "count": 43,
  },
}

for key in data:
  point = (
    Point("census")
    .tag("location", data[key]["location"])
    .field(data[key]["species"], data[key]["count"])
  )
  client.write(database=database, record=point)
  time.sleep(1) # separate points by 1 second

print("Complete. Return to the InfluxDB UI.")


Complete. Return to the InfluxDB UI.


## Execute a Simple Query
v3 supports SQL 

In [34]:
query = """SELECT *
FROM 'census'
WHERE time >= now() - interval '168 hours'
AND ('bees' IS NOT NULL OR 'ants' IS NOT NULL)"""

# Execute the query
table = client.query(query=query, database="Tables_Bucket", language='sql') 

# Convert to dataframe
df = table.to_pandas().sort_values(by="time")
column_names = df.columns
print(column_names)
print(df)




Index(['ants', 'bees', 'location', 'time'], dtype='object')
    ants  bees  location                          time
0    NaN  23.0   Klamath 2023-09-17 15:50:32.816591687
6   30.0   NaN  Portland 2023-09-17 15:50:34.151910341
1    NaN  28.0   Klamath 2023-09-17 15:50:35.335547781
7   32.0   NaN  Portland 2023-09-17 15:50:36.595085749
2    NaN  29.0   Klamath 2023-09-17 15:50:37.815063826
8   40.0   NaN  Portland 2023-09-17 15:50:38.975112553
3    NaN  23.0   Klamath 2023-09-17 19:48:09.639242780
9   30.0   NaN  Portland 2023-09-17 19:48:10.808677735
4    NaN  28.0   Klamath 2023-09-17 19:48:12.002052431
10  32.0   NaN  Portland 2023-09-17 19:48:13.156166514
5    NaN  29.0   Klamath 2023-09-17 19:48:14.321562173
11  40.0   NaN  Portland 2023-09-17 19:48:15.539494349
12   NaN  25.0   Klamath 2023-09-23 20:35:34.764041824
15  32.0   NaN  Portland 2023-09-23 20:35:36.123439256
13   NaN  28.0   Klamath 2023-09-23 20:35:37.362155251
16  36.0   NaN  Portland 2023-09-23 20:35:38.603744675
14   

## Advance Query
Sep 17 - doesn't work. Not sure why as it is copy paste from the demo. 
I suspect the fact I had to use "sql" as a language to make it work, and not "influxql" is the root cause. The documentation says using ```import influxdb_client_3 as InfluxDBClient3``` with influxql but it doesn't work.    

Trying to implement ```GROUP BY``` using: https://docs.influxdata.com/influxdb/v1/query_language/explore-data/#the-group-by-clause 

In [47]:
## Execute Aggregate Queries. The first one works
query = """
SELECT  location, max(time), avg(census.ants)
FROM "census"
WHERE time >= now() - interval '1 hour'
AND (ants IS NOT NULL)
GROUP BY location
"""



# Execute the query
table = client.query(query=query, database="Tables_Bucket", language='sql') 

# Convert to dataframe
df = table.to_pandas()#.sort_values(by='time')
print(df)

   location              MAX(census.time)  AVG(census.ants)
0  Portland 2023-09-23 20:35:40.936207328              37.0


You can always use a good olf Flux to run a GROUP BY query.

In [51]:
%%bash
influx query \
'from(bucket: "Tables_Bucket")
  |> range(start: -1h)
  |> filter(fn: (r) => r._measurement == "airSensors" and r._field == "humidity")
  |> aggregateWindow(every: 5m, fn: max)'



## Delete a bucket


In [ ]:
import requests

# InfluxDB API endpoint
base_url = "http://localhost:8086"
org = "your_organization"  # Replace with your organization name
bucket = "your_bucket"      # Replace with the name of the bucket you want to delete

# Authentication token (if required)
token = "your_authentication_token"  # Replace with your authentication token, if needed

# Construct the URL for deleting the bucket
url = f"{base_url}/api/v2/buckets/{org}/{bucket}"

# Headers for the request (include the authentication token if required)
headers = {
    "Authorization": f"Token {token}" if token else "",
}

# Send the DELETE request to delete the bucket
response = requests.delete(url, headers=headers)

# Check the response status code
if response.status_code == 204:
    print(f"Bucket '{bucket}' deleted successfully.")
else:
    print(f"Failed to delete bucket '{bucket}'. Status code: {response.status_code}")
    print(response.text)
